<a href="https://colab.research.google.com/github/alexlautw9527/for_notebook/blob/main/TRMSFMR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install transformers


In [25]:
import torch
import pandas as pd

In [ ]:
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')
model = AutoModel.from_pretrained('bert-base-chinese')

In [135]:

## 獨立對待
sentences = [
  '基金類別生技股',
  '基金類別全球股市',
  '基金類別新興市場債',
  '基金類別公司債',
  '基金類別科技股',
  '基金類別全球債',
  '基金類別美國公債',
  '美國公債',
  '人情債怎麼還',
  '漲跌幅震盪',
  '均線黃金交叉',
  '股市大多頭',
  '小狗','小貓','人類']


context1 = ['暴雷一時爽', '杰哥']
context2 = ['一直暴雷一直爽', '我看你是不懂哦']

In [52]:
tokenizer.decode(tokenizer(sentences)['input_ids'][1])

'[CLS] 定 期 定 額 發 大 財 [SEP]'

In [53]:
## 可用於問答or推論任務

tokenizer.decode(tokenizer(context1, context2)['input_ids'][0])

'[CLS] 暴 雷 一 時 爽 [SEP] 一 直 暴 雷 一 直 爽 [SEP]'

In [144]:
sentences

['基金類別生技股',
 '基金類別全球股市',
 '基金類別新興市場債',
 '基金類別公司債',
 '基金類別科技股',
 '基金類別全球債',
 '基金類別美國公債',
 '美國公債',
 '人情債怎麼還',
 '漲跌幅震盪',
 '均線黃金交叉',
 '股市大多頭',
 '小狗',
 '小貓',
 '人類']

In [145]:
## return_tensors="pt" 才能放進model
inputs = tokenizer(sentences, padding=True, return_tensors="pt")


In [ ]:
## 單純forward pass 沒要計算梯度
with torch.no_grad():
  outputs = model(**inputs)

In [151]:
x = outputs['last_hidden_state']

In [153]:
np.savetxt('bert_mkt_emb.tsv', x[:,-5:-1].mean(axis=1).cpu().detach().numpy(), delimiter="\t")


In [155]:
pd.DataFrame(sentences).to_csv('bert_mkt_emb_meta.tsv', header=False, index=False, sep="\t")

In [156]:
from transformers import pipeline

text_generator = pipeline("text-generation")
print(text_generator("I am a evil,", max_length=50, do_sample=False))

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I am a evil, evil, evil, evil, evil, evil, evil, evil, evil, evil, evil, evil, evil, evil, evil, evil, evil, evil, evil, evil, evil, evil, evil, evil'}]


In [161]:
from transformers import pipeline
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")

conversation = '''
Think of ETFs as buckets that hold a collection of securities, like stocks and bonds. Because ETFs are made up of these multiple assets, they provide investors instant diversification. When an investor purchases a share of an ETF, their money is spread across different investments. This differs from stocks where you buy shares of just a single company.

ETFs typically mimic a market index like the S&P 500. Since ETF performance is usually based on an index — meaning they follow the ups and downs of said index — most are passively managed investments and thus likely have lower fees than mutual funds. Mutual funds, on the other hand, want to beat the market’s performance and are thus managed by a fund manager, who’s actively choosing the investments.

Similar to stocks, ETFs can be bought and sold on an exchange throughout the day, and investors can even earn dividends depending on the type of index the fund tracks.

Should you invest in ETFs?
Since ETFs offer built-in diversification and don’t require large amounts of capital in order to invest in a range of stocks, they are a good way to get started. You can trade them like stocks while also enjoying a diversified portfolio.
'''

summarizer(conversation)


[{'summary_text': 'ETFs are buckets that hold a collection of securities, like stocks and bonds. They are passively managed and have lower fees than mutual funds. Similar to stocks, they can be bought and sold on an exchange throughout the day and investors can earn dividends depending on the type of index the fund tracks.'}]

In [ ]:
ETFs are buckets that hold a collection of securities, like stocks and bonds. 
They are passively managed and have lower fees than mutual funds. 
Similar to stocks, they can be bought and sold on an exchange throughout the day and investors can earn dividends depending on the type of index the fund tracks